# Real world Data

In this dataset, we have the production of a numerous amount of Chinese firm by city and industry. In order the create a balanced panel at the city-industry-year level, we need to create a squared matrix. It means, we need to have a dataframe with all the industries available by city and year. 

## Objective:

- Create a balanced panel data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("dataPandasClass_china.gz")
df.shape

In [ ]:
df.head()

In [ ]:
df['SOE'].unique()

In [ ]:
df.sort_values(['year', 'cityen', 'indu_2', 'SOE']).head()

In [ ]:
to_group_reshape = [

    'cityen',
    'year', 'indu_2', 'SOE'
]

In [ ]:
df_drop = df.drop(columns = 'geocode4_corr')
df_drop.head()

In [ ]:
df_drop.pivot_table(index = ['cityen', 'indu_2'],
         columns ='year',
         #values = 'output'
        ).head()

In [ ]:
df_drop.to_csv('test.csv')

In [ ]:
df_perc = (df_drop
           .groupby(['indu_2', 'SOE',  'year'])
           .sum()
           .groupby(level = [0, 1])
           .pct_change().reset_index()
          )
#).stack()
df_perc.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.catplot(x="year",
            y="output",
            col="indu_2",
            data=df_perc.loc[lambda x:x['SOE'] == 'SOE'], 
            kind="bar",
            col_wrap=5,
           )

In [ ]:
sns.catplot(x="SOE",
            y="output",
            col="indu_2",
            hue = 'year',
            data=df_perc, 
            kind="bar",
            col_wrap=5,
           )

In [ ]:
(df
 .set_index(to_group_reshape)
 .unstack(-1, fill_value=0)
 .stack()
 .unstack(-2, fill_value=0)
 .stack()
 .reset_index()
 )
